<a href="https://colab.research.google.com/github/nichakornchaisuwan/Project_Boneage/blob/main/lightning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pytorch-lightning

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 825.8/825.8 KB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 512.4/512.4 KB 31.2 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
from PIL import Image

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms

import pytorch_lightning as pl
#from pytorch_lightning.logging import TensorBoardLogger
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks.early_stopping import EarlyStopping

In [3]:
import re
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.checkpoint as cp
from collections import OrderedDict
import torch.utils.model_zoo as model_zoo
from torch.utils.model_zoo import load_url as load_state_dict_from_url

In [4]:
from google.colab import drive # เชื่อม drive ของเรา ถ้าเชื่อมสำเร็จจะขึ้นคำว่าMounted at /content/drive 
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import os # os จัดการเกี่ยวกับไฟล์ต่างๆ ในโฟล์เดอร์

In [6]:
path = '/content/drive/My Drive/Project_Boneage'  # ชี้ว่า data เราอยู่โฟล์เดอร์ไหน

In [7]:
train_img = '/content/drive/My Drive/Project_Boneage/boneage_training_dataset/'
val_img = '/content/drive/My Drive/Project_Boneage/boneage_validation_dataset/'
test_img = '/content/drive/My Drive/Project_Boneage/boneage-test-dataset/'

In [8]:
train = pd.read_csv(path+"/train.csv")
validation = pd.read_csv(path+"/validation.csv")
test = pd.read_csv(path+"/boneage-test-dataset.csv")


In [9]:
train

,id,boneage,male
0,1377,180,False
1,1378,12,False
2,1379,94,False
3,1380,120,True
4,1381,82,False
...,...,...,...
10084,13259,94,False
10085,13261,94,False
10086,13274,94,False
10087,13286,94,False


In [10]:
train['id'] = train['id'].apply(lambda x: str(x)+'.png')
train['boneage'] = train['boneage'].apply(lambda x: str(x))
train['male'] = train['male'].apply(lambda x: 0 if x else 1)

train = pd.DataFrame(train)
train.rename(columns={'male':'gender'},inplace=True)

In [11]:
train

,id,boneage,gender
0,1377.png,180,1
1,1378.png,12,1
2,1379.png,94,1
3,1380.png,120,0
4,1381.png,82,1
...,...,...,...
10084,13259.png,94,1
10085,13261.png,94,1
10086,13274.png,94,1
10087,13286.png,94,1


In [12]:
validation['id'] = validation['id'].apply(lambda x: str(x)+'.png')
validation['boneage'] = validation['boneage'].apply(lambda x: str(x))
validation['male'] = validation['male'].apply(lambda x: 0 if x else 1)

validation = pd.DataFrame(validation)
validation.rename(columns={'male':'gender'},inplace=True)

In [13]:
validation

,id,boneage,gender
0,10186.png,10,0
1,10758.png,15,1
2,10860.png,180,1
3,10877.png,180,1
4,11008.png,180,1
...,...,...,...
2517,15605.png,50,1
2518,15606.png,113,1
2519,15608.png,55,1
2520,15609.png,150,0


In [14]:
test

,id,boneage,male
0,4360,169,True
1,4361,170,True
2,4362,73,True
3,4363,153,True
4,4364,135,True
...,...,...,...
195,4555,133,False
196,4556,129,False
197,4557,168,False
198,4558,136,False


In [15]:
test['id'] = test['id'].apply(lambda x: str(x)+'.png')
#test['boneage'] = test['boneage'].apply(lambda x: str(x))
test['male'] = test['male'].apply(lambda x: 0 if x else 1)

test = pd.DataFrame(test)
test.rename(columns={'male':'gender'},inplace=True)



In [16]:
test

,id,boneage,gender
0,4360.png,169,0
1,4361.png,170,0
2,4362.png,73,0
3,4363.png,153,0
4,4364.png,135,0
...,...,...,...
195,4555.png,133,1
196,4556.png,129,1
197,4557.png,168,1
198,4558.png,136,1


#clone

##ลองดึง Inception_V3 จาก github ตัวเอง

In [17]:
#Local imports
#from Project_Boneage.backbone.inception_v3 import InceptionV3

##ลองดึง Inception_V3 จาก github ของ Bonet มาใช้ก่อน

In [18]:
! git clone https://github.com/BCV-Uniandes/Bonet.git
!cd Bonet


Cloning into 'Bonet'...
remote: Enumerating objects: 52, done.
remote: Total 52 (delta 0), reused 0 (delta 0), pack-reused 52
Unpacking objects: 100% (52/52), 136.62 KiB | 5.69 MiB/s, done.


In [19]:
# Local imports
from Bonet.models.backbone.inception_v3 import InceptionV3

In [20]:
start_epoch = 0
num_epochs = 2
lr=0.0001
patience = 2 
batch_size=20
num_workers=0
num_gpus = 1
gpus = 0
max_epochs = 2

In [21]:
#def __getitem__(self, idx):
#    image = transform(Image.open(self.df.Image[idx]).convert("RGB"))
#    label = self.encode[self.df.Finding[idx]]
#    return image, label

#Creating a custom PyTorch Dataset

In [22]:
class ImageDataset(Dataset):
    """Tabular and Image dataset."""

    def __init__(self, DataFrame, image_dir):
        self.image_dir = image_dir
        #self.pickle_file = pickle_file
        #self.frame = frame
        #self.table = train
        #self.table = validation
        #self.table = test
        self.DataFrame = DataFrame
        self.table = pd.DataFrame(DataFrame)


    def __len__(self):
        return len(self.table)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        table = self.table.iloc[idx, 0:]

        y = table["boneage"]

        image = Image.open(f"{self.image_dir}/{table['id']}")
        image = np.array(image)
        #image = image[..., :3]

        image = transforms.functional.to_tensor(image)
        image = transforms.functional.resize(image, size=[500,500])
        # image = transforms.Compose([transforms.Resize((500, 500)),
        #                        transforms.RandomAffine(
        #                            20, translate=(0.2, 0.2),
        #                            scale=(1, 1.2)),
        #                        transforms.RandomHorizontalFlip(),
        #                        transforms.ToTensor()])

        table = table[["gender"]]
        table = table.tolist()
        table = torch.FloatTensor(table)

        return image, table, y

In [23]:
#table=pd.read_csv(path+"/train.csv")
#table

In [24]:
np.array(Image.open(f"{train_img}{train.iloc[0,0:]['id']}"))[..., :3].shape

(2044, 3)

In [25]:
ImageDataset(train, image_dir=f"{train_img}")

In [26]:
#import torchvision.models as models
#inception = models.inception_v3(init_weights=True)

In [27]:
#def conv_block(input_size, output_size):
#    block = nn.Sequential(
#        nn.Conv2d(input_size, output_size, (3, 3)), nn.ReLU(), nn.BatchNorm2d(output_size), nn.MaxPool2d((2, 2)),
#    )
#
#    return block

In [28]:
class BoNetModule(pl.LightningModule):

    def __init__(
        self, transform_input = False, lr = 0.0001, num_workers = 0, batch_size = 20
    ):

        super().__init__()
        self.lr = lr
        self.num_workers = num_workers
        self.batch_size = batch_size
 
        # Backbone
        self.inception_v3 = InceptionV3()

        # Gender
        self.gender = DenseLayer(1, 32)
        self.fc_1 = DenseLayer(100384, 1000)
        self.fc_2 = DenseLayer(1000, 1000)
        self.fc_3 = nn.Linear(1000, 1)

    def forward(self, x, y):
        x = self.inception_v3(x)
        y = self.gender(y)
        x = self.fc_1(torch.cat([x, y], 1))
        x = self.fc_2(x)
        x = self.fc_3(x)
        return x

    def training_step(self, batch, batch_idx):
      image, table, y = batch

      criterion = torch.nn.L1Loss()
      y_pred = torch.flatten(self(image, table))
      y_pred = y_pred.double()

      loss = criterion(y_pred, y)

      tensorboard_logs = {"train_loss": loss}
      return {"loss": loss, "log": tensorboard_logs}

    def validation_step(self, batch, batch_idx):
        image, table, y = batch

        criterion = torch.nn.L1Loss()
        y_pred = torch.flatten(self(image, table))
        y_pred = y_pred.double()

        val_loss = criterion(y_pred, y)

        return {"val_loss": val_loss}

    def validation_epoch_end(self, outputs):
        avg_loss = torch.stack([x["val_loss"] for x in outputs]).mean()
        tensorboard_logs = {"val_loss": avg_loss}
        return {"val_loss": avg_loss, "log": tensorboard_logs}

    def test_step(self, batch, batch_idx):
        image, table, y = batch

        criterion = torch.nn.L1Loss()
        y_pred = torch.flatten(self(image, table))
        y_pred = y_pred.double()

        test_loss = criterion(y_pred, y)

        return {"test_loss": test_loss}

    def test_epoch_end(self, outputs):
        avg_loss = torch.stack([x["test_loss"] for x in outputs]).mean()
        logs = {"test_loss": avg_loss}
        return {"test_loss": avg_loss, "log": logs, "progress_bar": logs}

    def setup(self, stage):

        image_datatrain = ImageDataset(train, image_dir=f"{train_img}")
        image_dataval = ImageDataset(validation, image_dir=f"{val_img}")
        image_datatest = ImageDataset(test, image_dir=f"{test_img}")

        #train_size = int(0.80 * len(image_data))
        #val_size = int((len(image_data) - train_size) / 2)
        #test_size = int((len(image_data) - train_size) / 2)

        #self.train_set, self.val_set, self.test_set = random_split(image_data, (train_size, val_size, test_size))
        self.train_set = image_datatrain
        self.val_set = image_dataval
        self.test_set = image_datatest

        
    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=(self.lr),)

    def train_dataloader(self):
        return DataLoader(self.train_set, batch_size=self.batch_size)

    def val_dataloader(self):
        return DataLoader(self.val_set, batch_size=self.batch_size)

    def test_dataloader(self):
        return DataLoader(self.test_set, batch_size=self.batch_size)
    

class DenseLayer(pl.LightningModule):

    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.linear = nn.Linear(in_channels, out_channels)

    def forward(self, x):
        x = self.linear(x)
        return F.relu(x, inplace=True)



In [29]:
ImageDataset(test, image_dir=f"{val_img}")

In [30]:
#ImageDataset(train, image_dir=f"{train_img}")

In [31]:
#image_datatrain = ImageDataset(train, image_dir=f"{train_img}")[0][0].size()
#image_datatrain

In [32]:
#image_dataval = ImageDataset(validation, image_dir=f"{val_img}")[0][0].size()
#image_dataval

In [33]:
#image_datatest = ImageDataset(test, image_dir=f"{test_img}")[0][0].size()
#image_datatest 

#Training

In [34]:
from fastai.vision.learner import Learner
from fastai.callback.schedule import lr_find

In [35]:
logger = TensorBoardLogger("lightning_logs", name="image_only")
early_stop_callback = EarlyStopping(monitor="val_loss",min_delta=5000,patience=7, verbose=False, mode="min")
    

In [36]:
model = BoNetModule()
#trainer = pl.Trainer(accelerator='cpu', logger=logger, callbacks=early_stop_callback)
trainer = pl.Trainer(accelerator='gpu', devices=1, logger=logger,max_epochs=3 )

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [37]:
trainer.fit(model)
trainer.test(model)

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name         | Type        | Params
---------------------------------------------
0 | inception_v3 | InceptionV3 | 21.8 M
1 | gender       | DenseLayer  | 64    
2 | fc_1         | DenseLayer  | 100 M 
3 | fc_2         | DenseLayer  | 1.0 M 
4 | fc_3         | Linear      | 1.0 K 
---------------------------------------------
123 M     Trainable params
0         Non-trainable params
123 M     Total params
492.688   Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

AttributeError: ignored

In [ ]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [ ]:
tensorboard --logdir=lightning_log
plot_it(trainer)

In [ ]:
%tensorboard --logdir logs

In [ ]:
%tensorboard --logdir logs/fit